# Astra target landing site example

## Derivation of Nozzle lift equation (fixed ascent rate)

An estimate of the bounds of Nozzle lift is required for scipy's differential evolution algorithm. We'll use a primitive approach, since the method calculates (integrates) the ascent rate equation anyway, and a penalty method will be used for ascent rates reaching outside the desired boundaries.

Force balance:

\begin{equation}
(m_{bal} + m_{pay} + m_{gas}) \frac{d^2z}{dt^2} = B - W - D = 0\\
B = W + D\\
V(\rho_{air} - \rho_{gas}) g = (m_{bal} + m_{pay})g + \frac{1}{2} \rho_{air} \left(\frac{dz}{dt}\right)^2 C_D A
\end{equation}

Then use balloon volume to get a 3rd order polynomial in terms of $r$

\begin{equation}
\frac{4}{3} \pi (\rho_{air} - \rho_{gas}) g r^3 - \frac{1}{2} \rho_{air} \left(\frac{dz}{dt}\right)^2 C_D \pi r^2 - (m_{bal} + m_{pay})g = 0
\end{equation}

Solve for r, and extract real positive root (should be unique: otherwise, raise an error). Then use the relation of gas mass (eqn 3 in "High-Altitude Gas Balloon Trajectory Prediction: A Monte Carlo Model") to obtain the nozzle lift:

\begin{equation}
m_{gas} = \rho^0_{gas} V^0 = \rho^0_{gas} \frac{L_N/g + m_{bal}}{\rho^0_{air} - \rho^0_{gas}}\\
L_N/g = \frac{4\pi}{3} r^3 (\rho_{air}-\rho_{gas}) - m_{bal} \hspace{1em} (kg)
\end{equation}

This is found in ```astra.flight_tools.nozzleLiftFixedAscent```, and has been validated with [habhub](http://habhub.org/calc/), which gives nozzle lift values of the same magnitude, and similar value (+- 10%)

## Ground distance equation

The Haversine formula is used to obtain ground distance $\Delta$ of the landing site from its target (assuming a spherical Earth), as

\begin{equation}
    \Delta = 2r\dot{}\mathrm{arcsin}\left( \sqrt{\mathrm{sin^2} \left( \frac{\phi_2 - \phi_1}{2}\right) + \mathrm{cos}(\phi_1) \mathrm{cos}(\phi_2) \mathrm{sin}^2 \left(\frac{\lambda_2 - \lambda_1}{2}\right)  }\right),
\end{equation}

where $\phi, \lambda$ refer to latitude and longitude respectively, and $r$ is the Earth's average radius ($=6371km$)

## 2D Optimization: Time + Nozzle Lift

In [ ]:
%load_ext autoreload
%autoreload
import astra
from astra.target_landing import targetFlight
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from scipy.interpolate import interp1d
from astra.weather import forecastEnvironment
import astra
import os
from astra.GFS import GFS_Handler
%matplotlib notebook

In [ ]:
targetLat = 51.077214
targetLon = -1.1866875398775423
targetElev = 137

# Set the optimisation weights used for [distance from target, cost, flight duration] respectively:
# Note that these are negative, because we'd like to a minimize both.
weights = (-1, -0.1, -0.)

# Use a previous forecast (allows offline development)
launch_datetime = datetime(2017,6,23, 15)
inputs = {}
# simEnvironment = forecastEnvironment(launchSiteLat=50.903824,      # deg
#                                      launchSiteLon=-1.63697,     # deg
#                                      launchSiteElev=114.0,           # m
#                                      dateAndTime=launch_datetime,
#                                      forceNonHD=True,
#                                      debugging=True,
#                                      requestSimultaneous=False  # False has better memory overhead
#                                     )
# # Set up the example input data files (from 22/06/2017, Daytona Beach)
# fileDict = {}
# for paramName in GFS_Handler.weatherParameters.keys():
#     fileDict[paramName] = os.path.abspath('./data/gfs_0p50_06z.ascii?{}[11:36][0:46][263:301][0:719]'.format(paramName))
# simEnvironment.loadFromNOAAFiles(fileDict)
# inputs['lanuchSiteForecasts'] = simEnvironment

simulator = targetFlight(start_dateTime=launch_datetime,
                 targetLat=targetLat,
                 targetLon=targetLon,
                 targetElev=targetElev,
                 launchSites=[(50.903824, -1.63697, 114.0)],
                 balloonModel='TA100',
                 balloonGasType="Helium",
                 nozzleLift=1,
                 trainEquivSphereDiam=0.1,
                 inflationTemperature=0.0,
                 payloadTrainWeight=0.38,
                 windowDuration=72,
                 weights=weights,
                 HD=False,
                 maxFlightTime=18000,
                 parachuteModel=None,
                 debugging=True,
                 log_to_file=False,
                 progress_to_file=False,
                 outputFile=os.path.join(''),
                 **inputs)

Check Weather Interpolators work over this range (i.e., ensure they are not constant/extrapolations)

In [ ]:
import copy
%autoreload
import astra

# Number of x-y points to use
Nx = 21
Ny = 21

simulator.balloonModel = 'TA100'
simulator.weights = (-1, -0.1, -0.)

# This will download the forecast, if it hasn't been loaded already
bestProfile_bf, X, Y,scores = simulator.bruteForce(Nx, Ny)
results_bf = copy.deepcopy(simulator.results)

In [ ]:
# Temperature visualation at the landing site, over the time window
import matplotlib.dates as mdates

simEnvironment = simulator.environment

# Plot weather as a function of time over the date range:
ts = np.linspace(0, simulator.windowDuration, 3001)
datetimes = [simEnvironment.dateAndTime + timedelta(hours=t) for t in ts]

Ps = [simEnvironment.getTemperature(simEnvironment.launchSiteLat,
                                        simEnvironment.launchSiteLon,
                                        simEnvironment.launchSiteElev, dt)
          for dt in datetimes]

fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(datetimes, Ps)
ax.set_ylabel("Temperature")
ax.set_xlabel("Time after start date (hours)")

xtick_locator = mdates.AutoDateLocator()
xtick_formatter = mdates.AutoDateFormatter(xtick_locator)
ax.xaxis.set_major_locator(xtick_locator)
ax.xaxis.set_major_formatter(xtick_formatter)
ax.set_xlim(datetimes[0], datetimes[-1])

fig.autofmt_xdate()

In [ ]:
# simulator.results contains the lexographically sorted pareto front of the
# non dominated flights (based on weighted distance vs cost)
import matplotlib.tri as mtri


def plotParetoFront(fitnesses, paretoFront):
    """Plots the points stored in the simulator.ParetoFront (containing the non-dominated
    Pareto efficient individuals).
    
    Notes
    -----
    * Only works for 2 or 3 objectives. 
    * Assumes that all fitnesses have the same number of values and weightings
    """
    nonzero_indices = np.nonzero(fitnesses[0].weights)[0]
    assert(len(nonzero_indices) >= 2),\
        "Input fitness array has less than 2 weighted objectives: no useful Pareto solutions."
        
    # Dictionary of weight_index : label pairs. Extract the axes labels for the non zero weighted vars
    axlabelsAvailable = {0: r'$\bar{\Delta} \cdot w_1$',
                       1: r'$\bar{m}_{gas} \cdot w_2$',
                       2: r'$\bar{t} \cdot w_3$'}
    axlabels = [axlabelsAvailable[idx] for idx in nonzero_indices]
    
    if len(nonzero_indices) == 2:
        fig = plt.figure()
        ax = fig.add_subplot(111)
        fs = list(zip(*[fitness.wvalues for fitness in fitnesses]))
        f1 = -np.array(fs[nonzero_indices[0]])
        f2 = -np.array(fs[nonzero_indices[1]])
        ax.plot(f1, f2, 'kx')
        pareto_vars = list(zip(*[profile.fitness.wvalues for profile in paretoFront]))
        pareto_array = -np.array([pareto_vars[nonzero_indices[0]], pareto_vars[nonzero_indices[1]]]).T
        pareto_array = pareto_array[np.argsort(pareto_array[:, 1])]
        ax.plot(pareto_array[:, 0], pareto_array[:, 1], 'ko--', mfc='none',
                markersize=16, label='Pareto front')

        # Find best (assumes that the weights are tuned to how import each parameter is)
        iBest = np.argmin(pareto_array[:,0] + pareto_array[:, 1])
        ax.plot(pareto_array[iBest, 0], pareto_array[iBest, 1], 'k*', markersize=12, label='Best')

        # Formatting
        ax.set_xlabel(axlabels[0])
        ax.set_ylabel(axlabels[1])
        ax.legend(loc='upper right')
        
    elif len(nonzero_indices) == 3:
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        f1, f2, f3 = zip(*[fitness.wvalues for fitness in fitnesses])
        f1 = -np.array(f1)
        f2 = -np.array(f2)
        f3 = -np.array(f3)
        ax.scatter(f1, f2, f3, label='Brute Force Points')
        pareto_x, pareto_y, pareto_z = zip(*[profile.fitness.wvalues for profile in paretoFront])
        pareto_array = -np.array([pareto_x, pareto_y, pareto_z]).T
    
        ax.scatter(pareto_array[:, 0], pareto_array[:, 1], pareto_array[:, 2])#, facecolors='k',
#                    label='Pareto points')
    
        # Plot the Pareto surface triangulation
        triang = mtri.Triangulation(pareto_array[:,0],pareto_array[:,1])
        ax.plot_trisurf(triang,pareto_array[:,2],color='red', alpha=0.5)

        # Find best (assumes that the weights are tuned to how important each parameter is)
        iBest = np.argmin(pareto_array[:,0] + pareto_array[:, 1])
        ax.plot([pareto_array[iBest, 0]], [pareto_array[iBest, 1]], [pareto_array[iBest, 2]], 'k*', markersize=12, label='Best')

        # Formatting
        ax.set_xlabel(axlabels[0])
        ax.set_ylabel(axlabels[1])
        ax.set_zlabel(axlabels[2])
        ax.legend(loc='upper right')        
    
    else:
        raise ValueError("Can't support plotting {}D Pareto front. Adjust weightings.".format(len(nonzero_indices)))
    
        return None

plotParetoFront(simulator.fitnesses, simulator.results)

In [ ]:
# Visualise the Pareto Front result paths (This may not be all that useful)
simulator.plotPaths3D()

In [ ]:
# Visualise the surface as a 3D surface contour:
%autoreload
from matplotlib import cm

fig1, ax1 = simulator.plotObjectiveContours3D(
    X, Y, scores.T, appendLabel=' Brute Force', cmap=cm.viridis, bestProfile=bestProfile_bf)

In [ ]:
%autoreload
import astra

# fig3, ax3 = simulator.plotObjectiveContours(X, Y, scores.T,
#     appendLabel=' Brute Force', bestProfile=bestProfile_bf)

# To manually set the levels used in the plot, pass the levels as an additional arg:
levels = np.linspace(0.0, 0.16, 9)
print(levels)
fig3, ax3 = simulator.plotObjectiveContours(X, Y, scores.T,
    appendLabel=' Brute Force', bestProfile=bestProfile_bf, levels=levels)

## Nelder Mead Simplex Algorithm

Running the code here will update the contour plot above.

In [ ]:
%autoreload
import astra

In [ ]:
# Optimize with Scipy:

res_scipy = simulator.optimizeTargetLandingSite(x0=[0.5*simulator.windowDuration, 1.0],
                                                options={'xtol': 1e-6, "disp": True, 'maxiter':50})
bestProfile_scipy = simulator.bestProfile
results_scipy = simulator.results
Xs_scipy = simulator.Xs

In [ ]:
# Optimize with scipy differential evolution
simulator.Xs = Xs_scipy
simulator.plotObjectiveLocations(fig3, ax3, marker='rx', bestMarker='r*', 
                                   bestProfile=bestProfile_scipy, appendLabel=' Nelder-Mead')

In [ ]:
print(res_scipy)

## L-BFGS-B Algorithm

Commenting out the code here for now, as the optimization doesn't work so well...

In [ ]:
# from astra.flight_tools import nozzleLiftFixedAscent

# # Optimize with Scipy:
# start_timedelta = 0
# end_timedelta = simulator.windowDuration

# nozzleLiftLowerBound = nozzleLiftFixedAscent(simulator.minAscentRate,
#                 simulator._balloonWeight, simulator.payloadTrainWeight,
#                 simulator.environment.inflationTemperature,
#                 simulator.environment.getPressure(simulator.launchSiteLat,
#                                                   simulator.launchSiteLon,
#                                                   simulator.launchSiteElev,
#                                                   simulator.start_dateTime),
#                 simulator._gasMolecularMass, simulator.excessPressureCoeff,
#                 CD=(0.225 + 0.425)/2.)
# nozzleLiftUpperBound = nozzleLiftFixedAscent(simulator.maxAscentRate,
#                 simulator._balloonWeight, simulator.payloadTrainWeight,
#                 simulator.environment.inflationTemperature,
#                 simulator.environment.getPressure(simulator.launchSiteLat,
#                                                   simulator.launchSiteLon,
#                                                   simulator.launchSiteElev,
#                                                   simulator.start_dateTime),
#                 simulator._gasMolecularMass, simulator.excessPressureCoeff,
#                 CD=(0.225 + 0.425)/2.)

# res_lbfgsb = simulator.optimizeTargetLandingSite(x0=[0.5*simulator.windowDuration, 1.0],
#                                                  method='L-BFGS-B',
#                                                  bounds=([start_timedelta, end_timedelta],
#                                                          [nozzleLiftLowerBound, nozzleLiftUpperBound]),
#                                                  options={"disp": True, "maxfun":200},
#                                                  )
# bestProfile_lbfgsb = simulator.bestProfile
# results_lbfgsb = simulator.results
# Xs_lbfgsb = simulator.Xs

In [ ]:
# # Optimize with scipy differential evolution
# simulator.bestProfile = bestProfile_lbfgsb
# simulator.results = results_lbfgsb
# simulator.Xs = Xs_lbfgsb
# simulator.plotObjectiveLocations(fig3, ax3, marker='cx', bestMarker='c*', appendLabel=' L-BFGS-B')

In [ ]:
# print(res_lbfgsb)

## Scipy differential Evolution

In [ ]:
%autoreload
import astra

res_de = simulator.optimizeTargetLandingSite(method='DE', tol=1, maxiter=6)
results_de = simulator.results
bestProfile_de = simulator.bestProfile
Xs_de = simulator.Xs

In [ ]:
res_de

In [ ]:
simulator.results = results_de
simulator.bestProfile = bestProfile_de
simulator.Xs = Xs_de
simulator.plotObjectiveLocations(fig3, ax3, marker='kx', bestMarker='k*', appendLabel=' DE')

In [ ]:
fig3.savefig('Distance2D_NozzleLift_LaunchTime.pdf')

In [ ]:
print("Brute Force: delta={} km, nevals={}".format(bestProfile_bf.distanceFromTarget, Nx * Ny))
print("Differential Evolution: delta={} km, nevals={}".format(bestProfile_de.distanceFromTarget, res_de.nfev))
print("Nelder Mead: delta={} km, nevals={}".format(bestProfile_scipy.distanceFromTarget, res_scipy.nfev))

## Higher Dimensional Problem

### Slice with cutdown altitude

In [ ]:
import copy
%autoreload
import astra

# Number of x-y points to use
Nx = 6
Ny = 6

# Could also use 'floatingAltitude' as the sliceParam
bestProfile_slicing, X, Y, Z, distances = simulator.bruteForceSlice(Nx, Ny, sliceParam='cutdownAltitude',
                                                            Nslices=3, sliceBounds=(10e3, 18e3))
results_slicing = copy.deepcopy(simulator.results)



In [ ]:
for i, z in enumerate(Z):
    scores_slice = distances[:, :, i]
# #     levels = np.arange(0, 90, 10)
    fig, ax = simulator.plotObjectiveContours(X, Y, scores_slice.T,
        appendLabel=' Brute Force')
    ax.set_title(r"Cutdown Altitude: ${}m$".format(z))
#     fig.savefig('cutdownAltitude_{}_contour.pdf'.format(z))

## Optimization in higher dimensions

The input vector we will try to solve the following optimization problem

\begin{equation}
\mathrm{min}(f(\textbf{x})) \\
f(\mathbf{x}) = \mathrm{\Delta}(\textbf{x}) + p_0(\textbf{x})\\
p_0 = \begin{cases} 
    5 \times 10^6 & \text{if not landed} \\
    & 0 & \text{otherwise}
   \end{cases}\\
\textbf{x} = [t, L_N, i_f, h_f, t_f, i_c, h_c, D_\mathrm{burst}],\\
t \in [0, 72] \mathrm{hours}\\
L_N \propto \bar{\frac{dz}{dt}}, \hspace{1em} \bar{\frac{dz}{dt}} \in [1.5, 6] \mathrm{m/s}\\
i_f \in [0, 1] \\
h_f \in [h_{f0}, h_{f1}]\\
t_f \in [0, \infty]\\
i_c \in [0, 1]\\
h_c \in [h_{c0}, h_{c1}]\\
D_\mathrm{burst} \in [D_\mathrm{burst0}, D_\mathrm{burst1}]
\end{equation}

where \Delta is the ground distance from the landing site obtained for a particular input vector $\bf{x}$, subscripts $f$ and $c$ denote floating and cutdown flight, $i_f, i_c$ are real variables bounded $\in [0, 1]$ that correspond to binary *on/off* behaviour, $h$ denotes altitude, $D_{burst}$ is the nominal burst diameter (continuous) varying between the bounds of a subset of chosen 'available' balloons, and $t_f$ is the duration  force the balloon to drop after a floating flight . Currently $h_{f0}, h_{f1}, h_{c0}, h_{c1}$ are user inputs for 'sensible' values of floating or cutdown flight altitude for a given mission, taking into account the nominal balloon burst altitude, local regulations and air traffic; however, this may be eligible for calculation in future.

Differential evolution was the most robust in the previous example, independant of the initial (conversely to Nelder-Mead), so we'll use it here.

**Notes**
* `cutdown` does not have to be activated ($i_c = 1$) for floating flight to begin decent after flight duration $t_c$.

In [ ]:
%autoreload
import astra

res_de_HD = simulator.optimizeTargetLandingSite(method='DE', useCutdown=True, useFloating=True,
                                             cutdownAltitudeBounds=(8e3, 16e3), floatingAltitudeBounds=(8e3, 16e3),
                                             flexibleBalloon=True,
                                             balloonModels=['TA100', 'TA200', 'HW200', 'HW800'],
                                             tol=1, maxiter=5, seed=62)
results_de_HD = simulator.results
bestProfile_de_HD = results_de_HD[0]
Xs_de_HD = simulator.Xs

In [ ]:
plotParetoFront(simulator.fitnesses, simulator.results)

In [ ]:
print(len(simulator.results))

In [ ]:
res_de_HD

In [ ]:
simulator.plotPaths3D()

## Testing

Random elements of code to go here....

In [ ]:
# This url appears to have problems in some circumstances - is it a cycle not available error, or something else?
url = "http://nomads.ncep.noaa.gov:9090/dods/gfs_0p50/gfs20170619/gfs_0p50_06z.ascii?hgtprs[10:35][0:46][263:301][0:719]"

from six.moves.urllib.request import urlopen

for i in range(20):
    HTTPresponse = urlopen(url)
    response = HTTPresponse.read().decode('utf-8')
    if response[0] == "<":
        print(response)

In [ ]:
import datetime
td = datetime.timedelta(seconds=666)
td.seconds

In [ ]:
balloonModels = 'TA100', 'HW1000', 'TA2000'
balloonsSelected = {k: balloons[k] for k in balloonModels}
balloonsSelected

In [ ]:
max(balloonsSelected, key=lambda k: balloonsSelected.get(k)[1])